In [22]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
import numpy as np
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 125)

url = 'https://raw.githubusercontent.com/KasperLoos/ConsultElement/main/data/gold_data/full_cleaned.csv'
response = requests.get(url)

# Save the content of the file
with open('full_cleaned.csv', 'wb') as file:
    file.write(response.content)

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('full_cleaned.csv')

In [23]:

df

,name,funding_total_usd,country_code,funding_rounds,founded_at,first_funding_at,last_funding_at,word_count,internal_link_count,image_count,has_team_page,has_product_page,funding_total_mean_imputed,time_before_first_funding,time_between_first_last_funding,founded_year,founded_month,funding_year,funding_month,has_website,sector,failed,dataset,average_funding
0,Kona Medical,73657688.0,USA,7,2009-01-01,2010-03-05,2015-02-09,366.0,47.0,10.0,0,0,7.365769e+07,428.0,1802.0,2009.0,1.0,2010.0,3.0,1,Healthcare and wellness,0,train,1.052253e+07
1,paraBebes.com,261560.0,ESP,1,2005-01-01,2005-01-01,2005-01-01,1197.0,14.0,84.0,0,0,2.615600e+05,0.0,0.0,2005.0,1.0,2005.0,1.0,1,Healthcare and wellness,0,train,2.615600e+05
2,Grannus,2000000.0,USA,1,2012-01-01,2012-01-01,2012-01-01,0.0,0.0,0.0,0,0,2.000000e+06,0.0,0.0,2012.0,1.0,2012.0,1.0,1,Other,0,train,2.000000e+06
3,Unconstrained Robotics,17000.0,other,1,NaN,NaN,NaN,0.0,0.0,0.0,0,0,1.667400e+04,0.0,0.0,unknown,unknown,unknown,unknown,0,Transport,1,train,1.700000e+04
4,Smarkets,2710000.0,GBR,4,2008-01-01,2008-01-05,2008-01-05,328.0,0.0,11.0,0,0,2.710000e+06,4.0,0.0,2008.0,1.0,2008.0,1.0,1,Entertainment,0,train,6.775000e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66363,Littlefeet Inc.,3000000.0,other,1,2002-01-11,2002-01-11,2002-01-11,0.0,0.0,0.0,0,0,3.000000e+06,0.0,0.0,2002.0,1.0,2002.0,1.0,0,Service Sector,1,test,3.000000e+06
66364,FoodFan,250000.0,USA,1,2011-01-01,2012-01-01,2012-01-01,68.0,0.0,0.0,0,0,2.500000e+05,365.0,0.0,2011.0,1.0,2012.0,1.0,1,(Social) Media & Communication,0,test,2.500000e+05
66365,EazyCoach,0.0,IND,1,2013-01-01,2013-01-01,2013-01-01,0.0,0.0,0.0,0,0,1.007059e+07,0.0,0.0,2013.0,1.0,2013.0,1.0,1,Retail & E-commerce,0,test,0.000000e+00
66366,MobileAware,6960000.0,USA,2,2000-01-01,2000-01-01,2008-11-08,53.0,9.0,18.0,0,0,6.960000e+06,0.0,3234.0,2000.0,1.0,2000.0,1.0,1,art & creative,0,test,3.480000e+06


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, make_scorer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import xgboost as xgb


# Custom penalty score
def custom_loss(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    fn = np.sum((y_true == 1) & (y_pred == 0))  # Missed failures
    fp = np.sum((y_true == 0) & (y_pred == 1))  # Incorrect failure predictions

    loss = 100 * fn + 1 * fp
    return -loss  # We return negative loss because GridSearchCV *maximizes* the score
custom_scorer = make_scorer(custom_loss, greater_is_better=True)


# Define the features (X) and the target (y)
X = df[['sector', 'funding_total_usd', 'country_code', 'has_team_page', 'has_product_page',
        'funding_rounds', 'time_before_first_funding', 'time_between_first_last_funding',
        'founded_year', 'founded_month', 'funding_year', 'funding_month', 'average_funding',
        'has_website', 'word_count', 'internal_link_count', 'image_count']]
y = df['failed']

# Define categorical columns
categorical_columns = ['sector', 'country_code', 'has_website', 'founded_year', 'founded_month', 'has_team_page', 'has_product_page', 'funding_year', 'funding_month']

# Create the column transformer for one-hot encoding
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough'  # Keeps the rest of the columns (like numeric ones)
)

# Split the dataset into training and testing sets based on the 'dataset' column
X_train = X[df['dataset'] == 'train']
X_test = X[df['dataset'] == 'test']
y_train = y[df['dataset'] == 'train']
y_test = y[df['dataset'] == 'test']

print(X_train.dtypes)

sector                              object
funding_total_usd                  float64
country_code                        object
has_team_page                        int64
has_product_page                     int64
funding_rounds                       int64
time_before_first_funding          float64
time_between_first_last_funding    float64
founded_year                        object
founded_month                       object
funding_year                        object
funding_month                       object
average_funding                    float64
has_website                          int64
word_count                         float64
internal_link_count                float64
image_count                        float64
dtype: object


In [25]:
# Create a pipeline with SMOTE or SMOTETomek and the classifier or just sklearn pipeline
from sklearn.model_selection import RandomizedSearchCV

from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', xgb.XGBClassifier(eval_metric='logloss', random_state=42))
])
"""
# Define the parameter grid
param_grid = {
    'classifier__max_depth': [3, 6, 10],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__subsample': [0.6, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.8, 1.0],
    'classifier__scale_pos_weight': [
        (len(y_train) - sum(y_train)) / sum(y_train)
    ],
    'classifier__gamma': [0, 0.1, 0.5],
    'classifier__min_child_weight': [1, 3, 5],
    'classifier__reg_alpha': [0, 0.5, 1],
    'classifier__reg_lambda': [1, 1.5, 2]
}


# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=3,
    scoring=custom_scorer,
    verbose=2,
    return_train_score=True,
    n_jobs=-1  # Parallelize across all CPU cores
)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predict
y_pred = best_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
loss = custom_loss(y_test, y_pred)
print(f"Custom penalty loss: {-loss}")

"""
param_dist = {
    'classifier__max_depth': [3, 5, 7, 10],
    'classifier__learning_rate': np.linspace(0.01, 0.3, 10),
    'classifier__n_estimators': [100, 200, 300, 500],
    'classifier__subsample': [0.6, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.8, 1.0],
    'classifier__scale_pos_weight': [
        (len(y_train) - sum(y_train)) / sum(y_train)
    ],
    'classifier__gamma': [0, 0.1, 0.3, 0.5],
    'classifier__min_child_weight': [1, 3, 5],
    'classifier__reg_alpha': [0, 0.1, 0.5, 1],
    'classifier__reg_lambda': [1, 1.5, 2]
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=50,  # You can adjust this to balance performance and runtime
    scoring=custom_scorer,  # Change to your preferred metric
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)
random_search.fit(X_train, y_train)



Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('onehot',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               ['sector',
                                                                                'country_code',
                                                                                'has_website',
                                                                                'founded_year',
                                                                                'founded_month',
                                                                                'has_team_page',
                                                                                'has_product_page',
                                                                                'funding_year',
                                                                                'funding_month'])])),
                                             ('classifier',
                                              XGBC...
                                        'classifier__max_depth': [3, 5, 7, 10],
                                        'classifier__min_child_weight': [1, 3,
                                                                         5],
                                        'classifier__n_estimators': [100, 200,
                                                                     300, 500],
                                        'classifier__reg_alpha': [0, 0.1, 0.5,
                                                                  1],
                                        'classifier__reg_lambda': [1, 1.5, 2],
                                        'classifier__scale_pos_weight': [9.780507614213198],
                                        'classifier__subsample': [0.6, 0.8,
                                                                  1.0]},
                   random_state=42,
                   scoring=make_scorer(custom_loss, response_method='predict'),
                   verbose=1)

In [27]:
# Print the best parameters
print("Best Parameters:", random_search.best_params_)

# Train the model with the best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

# Predict
y_pred = best_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Custom loss (assuming you defined custom_loss elsewhere)
loss = custom_loss(y_test, y_pred)
print(f"Custom penalty loss: {-loss}")

Best Parameters: {'classifier__subsample': 0.6, 'classifier__scale_pos_weight': 9.780507614213198, 'classifier__reg_lambda': 1, 'classifier__reg_alpha': 0, 'classifier__n_estimators': 200, 'classifier__min_child_weight': 3, 'classifier__max_depth': 3, 'classifier__learning_rate': np.float64(0.10666666666666666), 'classifier__gamma': 0.5, 'classifier__colsample_bytree': 0.8}
Accuracy: 0.6964
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.70      0.81     11961
           1       0.20      0.68      0.31      1313

    accuracy                           0.70     13274
   macro avg       0.58      0.69      0.56     13274
weighted avg       0.88      0.70      0.76     13274

Custom penalty loss: 45511


In [ ]:
feature_importances = best_model.named_steps['classifier'].feature_importances_
import matplotlib.pyplot as plt
import numpy as np

# Assuming `column_transformer` has transformed the features, get the column names
feature_names = best_model.named_steps['preprocessor'].get_feature_names_out()

# Sort feature importances in descending order
sorted_idx = np.argsort(feature_importances)[::-1]

# Plot the top 10 most important features
plt.figure(figsize=(10, 6))
plt.barh(feature_names[sorted_idx][:10], feature_importances[sorted_idx][:10])
plt.xlabel('Feature Importance')
plt.title('Top 10 Feature Importances')
plt.gca().invert_yaxis()  # To display the most important feature at the top
plt.show()

In [ ]:
# Get predicted probabilities
y_proba = best_model.predict_proba(X_test)[:, 1]

# Use a lower threshold
from sklearn.metrics import classification_report
threshold = 0.3
y_pred_adj = (y_proba > threshold).astype(int)

print(classification_report(y_test, y_pred_adj))